### Modelo

In [13]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
model = ChatOpenAI(model="gpt-4o")

### Tools

In [44]:
from langchain.agents import Tool

# Tool 1
def tool_1_template(pregunta):
    return f"""
            Conoces en detalle las velocidades radiales de las estrellas en la galaxia. Responde la pregunta: {pregunta}
            """

def seller_vehicles(input=""):
    prompt = tool_1_template(input)
    response = model(prompt)
    return response

seller_tool = Tool(
    name = "velocidades_estrellas",
    func = seller_vehicles,
    description = "Conoces las estrellas de la galaxia y te especializas en velocidades radiales. Responde como si se lo explicaras a un niño interesado en ciencia."
)

# Tool 2

In [ ]:
¿Cuál es la velocidad radial de una estrella en relación con el Sol?

### Agente

In [47]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key = "chat_history",
    k = 3,
    return_messages=True
)

tools = [seller_tool]

from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent = "chat-conversational-react-description",
    tools = tools,
    llm = model,
    verbose = True,
    max_iterations = 3,
    early_stopping_method = "generate",
    memory = memory
)

### Otros

In [48]:
Input = "¿Cuál es la velocidad radial de una estrella en relación con el Sol?"
Output = conversational_agent(Input)



> Entering new AgentExecutor chain...
```json
{
    "action": "velocidades_estrellas",
    "action_input": "velocidad radial de una estrella en relación con el Sol"
}
```
Observation: content='La velocidad radial de una estrella es la componente de su velocidad que apunta directamente hacia o se aleja de un observador, en este caso, el Sol. Se mide a lo largo de la línea de visión entre el observador y la estrella. Esta velocidad se determina principalmente utilizando el efecto Doppler, que provoca un desplazamiento en las líneas espectrales de la estrella.\n\nSi la estrella se está acercando al Sol, sus líneas espectrales se desplazan hacia el azul (conocido como corrimiento al azul), mientras que si se está alejando, se desplazan hacia el rojo (corrimiento al rojo). Estas mediciones permiten a los astrónomos calcular la velocidad radial de la estrella con respecto al Sol.\n\nLa velocidad radial es solo una parte de la velocidad espacial total de una estrella, que también incluye co

In [49]:
print(f"Humano: {Input}")
print("="*10)
print(f"AI: {Output['output']}")
print("="*10)

Humano: ¿Cuál es la velocidad radial de una estrella en relación con el Sol?
AI: La velocidad radial de una estrella es la parte de su velocidad que se dirige directamente hacia o se aleja de un observador, como el Sol. Esta velocidad se mide a lo largo de la línea de visión entre el observador y la estrella. Para determinarla, se utiliza el efecto Doppler, que causa un cambio en las líneas espectrales de la estrella: si la estrella se acerca al Sol, sus líneas espectrales se desplazan hacia el azul, y si se aleja, se desplazan hacia el rojo. Estas mediciones permiten calcular la velocidad radial de la estrella con respecto al Sol. Cabe mencionar que la velocidad radial es solo una parte de la velocidad total de una estrella, que también incluye el movimiento perpendicular a la línea de visión.


In [50]:
Output['chat_history']

[]